In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
transformer = transforms.Compose([
                                  transforms.RandomPerspective(distortion_scale=0.6,p=0.5), transforms.RandomRotation(degrees=(0,180)),
                                  transforms.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)), transforms.RandomInvert(p=0.2),
                                  transforms.RandomAdjustSharpness(0, p=0.25), transforms.RandomAdjustSharpness(2, p=0.25),
                                  transforms.RandomAutocontrast(p=0.2), transforms.RandomEqualize(p=0.1),
                                  transforms.Resize((150,150)),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.5,0.5,0.5], 
                                                        [0.5,0.5,0.5])
])

In [ ]:
#Dataloader
train_path = "/content/drive/MyDrive/Colab Notebooks/new_clean_emotions_dataset_v0.1/train/"
test_path = "/content/drive/MyDrive/Colab Notebooks/new_clean_emotions_dataset_v0.1/train/"

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size = 32, shuffle = True
)


In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
classes

['angry',
 'confused',
 'crying',
 'fear',
 'happy',
 'puzzled',
 'sad',
 'scared',
 'shy']

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=9):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output


In [ ]:
net=ConvNet(num_classes=9)
model = net

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10

In [ ]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)
train_dataset = train_count
validation_dataset = test_count

1354 1354


In [ ]:
def Train(epochs,train_loader,val_loader,criterion,optmizer,device, writer):
    '''
    Training Loop
    '''
    best_accuracy = 0
    print("===================================Start Training===================================")
    for e in range(epochs):
        train_loss = 0
        validation_loss = 0
        train_correct = 0
        val_correct = 0
        # Train the model  #
        net.train()
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            try:
              optmizer.zero_grad()
            except Exception:
              pass
            
            outputs = net(data)
            
            loss = criterion(outputs,labels)
            loss.backward()
            optmizer.step()
            train_loss += loss.item()
            _, preds = torch.max(outputs,1)
            train_correct += torch.sum(preds == labels.data)

        #validate the model#
        net.eval()
        for data,labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            val_outputs = net(data)
            val_loss = criterion(val_outputs, labels)
            validation_loss += val_loss.item()
            _, val_preds = torch.max(val_outputs,1)
            val_correct += torch.sum(val_preds == labels.data)

        train_loss = train_loss/train_dataset
        train_acc = train_correct.double() /train_dataset
        validation_loss =  validation_loss /validation_dataset
        val_acc = val_correct.double() /validation_dataset
        print('Epoch: {} \tTraining Loss: {:.8f} \tValidation Loss {:.8f} \tTraining Acuuarcy {:.3f}% \tValidation Acuuarcy {:.3f}%'
                                                           .format(e+1, train_loss,validation_loss,train_acc * 100, val_acc*100))
        
        if val_acc>best_accuracy:
          torch.save(net.state_dict(),'/content/drive/MyDrive/Colab Notebooks/best_checkpoint.model')
          best_accuracy=val_acc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/fer2013_experiment_1')
criterion= nn.CrossEntropyLoss()

In [ ]:
Train(100,train_loader,test_loader,criterion,optimizer,device,writer)

===================================Start Training===================================
Epoch: 1 	Training Loss: 0.44504537 	Validation Loss 0.13421032 	Training Acuuarcy 66.396% 	Validation Acuuarcy 75.554%
Epoch: 2 	Training Loss: 0.16755096 	Validation Loss 0.29400160 	Training Acuuarcy 66.248% 	Validation Acuuarcy 24.151%
Epoch: 3 	Training Loss: 0.16138744 	Validation Loss 0.21530649 	Training Acuuarcy 65.140% 	Validation Acuuarcy 53.840%
Epoch: 4 	Training Loss: 0.11355878 	Validation Loss 0.07430257 	Training Acuuarcy 67.134% 	Validation Acuuarcy 79.764%
Epoch: 5 	Training Loss: 0.05365181 	Validation Loss 0.05758102 	Training Acuuarcy 72.526% 	Validation Acuuarcy 80.724%
Epoch: 6 	Training Loss: 0.04456863 	Validation Loss 0.03958541 	Training Acuuarcy 77.105% 	Validation Acuuarcy 80.724%
Epoch: 7 	Training Loss: 0.03298505 	Validation Loss 0.03235780 	Training Acuuarcy 80.059% 	Validation Acuuarcy 80.724%
Epoch: 8 	Training Loss: 0.03027520 	Validation Loss 0.03014950 	Training A